# ExMatrix
### Como usar o sistema de recomendação.

- Author: Israel Oliveira [\[e-mail\]](mailto:'Israel%20Oliveira%20'<prof.israel@gmail.com>)

In [1]:
import sys
sys.path.insert(1, 'src/')
from train import *
from recommender import Recommender 

from tqdm import tqdm
import pandas as pd
from time import time

In [2]:
%load_ext watermark

In [3]:
# Run this cell before close.
%watermark -d --iversion -b -r -g -m -v

pandas 1.0.5
numpy  1.19.0
2020-07-27 

CPython 3.7.8
IPython 7.16.1

compiler   : GCC 8.3.0
system     : Linux
release    : 5.4.0-7634-generic
machine    : x86_64
processor  : 
CPU cores  : 8
interpreter: 64bit
Git hash   : 90851ad626ecf179ca5598e03ce4034e1a78e1a7
Git repo   : https://github.com/ysraell/aceleradev_private.git
Git branch : master


# Treinando o modelo.
Dataset em `./data/`, salvando o modelo em `./model/`.

In [4]:
t = time()
train('data/','model/')
dt = time() - t
print("Tempo total de treino: {:.2f} min.".format(dt/60))

2020-07-27 01:24:58.316 | INFO     | utils:load_dataset:82 - Carregando dataset de treino...
2020-07-27 01:25:09.016 | INFO     | utils:load_dataset:91 - ...pronto!
2020-07-27 01:25:09.017 | INFO     | utils:feat_proc:128 - Processando as features...
2020-07-27 01:25:44.004 | INFO     | utils:feat_proc:163 - ...pronto!
2020-07-27 01:25:44.386 | INFO     | train:train:35 - Treinando e modelo...
2020-07-27 01:25:44.599 | INFO     | model:fit:69 - Processando valores.
2020-07-27 01:25:52.209 | INFO     | model:fit:73 - Fatorizando.
/usr/local/lib/python3.7/site-packages/sklearn/decomposition/_nmf.py:1077: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  " improve convergence." % max_iter, ConvergenceWarning)
2020-07-27 01:35:21.182 | INFO     | model:fit:78 - Matriz pronta.
2020-07-27 01:35:21.183 | INFO     | train:train:37 - ...pronto.
2020-07-27 01:35:21.184 | ERROR    | train:train:45 - Erro durante o treino: [Errno 2] No such file or

Traceback (most recent call last):
  File "src/train.py", line 40, in train
    model, open(model_file, "wb"), protocol=pickle.HIGHEST_PROTOCOL,
FileNotFoundError: [Errno 2] No such file or directory: 'model/model_default.pkl'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-4-235a63355d0f>", line 2, in <module>
    train('data/','model/')
  File "src/train.py", line 50, in train
    sys.exit(exitStatus)
SystemExit: 255

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/IPython/core/ultratb.py", line 1169, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/usr/local/lib/python3.7/site-packages/IPython/co

TypeError: object of type 'NoneType' has no len()

# Carrega o datset para demonstração.

In [6]:
data, portfolios = load_dataset(path_data = 'data/', train_test_merged = True)

2020-07-26 22:24:35.946 | INFO     | utils:load_dataset:82 - Carregando dataset de treino...
2020-07-26 22:24:45.903 | INFO     | utils:load_dataset:91 - ...pronto!
2020-07-26 22:24:45.905 | INFO     | utils:load_dataset:95 - Carregando dataset de teste...
2020-07-26 22:24:45.920 | INFO     | utils:load_dataset:104 - ...pronto!


# Carrega o modelo.

In [7]:
model = Recommender('model/')

2020-07-26 22:24:47.521 | INFO     | recommender:__init__:33 - Modelo carregado


#### Exemplo de um ID:

In [8]:
portfolios.loc[0]

id    09e95c1a84049001d086470a8f320a19b076f955a89122...
P                                                     1
Name: 0, dtype: object

In [9]:
portfolios.loc[portfolios.id == model.For([portfolios.id.loc[0]])[0]]

,id,P
473,b9ecb6629e14d5e760685521d1394e6ed76a563f770e61...,1


## Recomendações 1-K
- Entra `1` ID e saem `K` recomendações.

In [10]:
def one_N(N,K):
    tmp = {1: [], 2: []}
    if N > 0:
        for row in tqdm(portfolios.sample(N,random_state=2).iterrows(), total=N):
            recs = model.For([row[1].id],k=K)
            tmp[row[1].P].append(any([x in portfolios.loc[portfolios.P == row[1].P].id.to_list() for x in recs])*1)
    else:
        for row in tqdm(portfolios.iterrows(), total=portfolios.shape[0]):
            recs = model.For([row[1].id],k=K)
            tmp[row[1].P].append(any([x in portfolios.loc[portfolios.P == row[1].P].id.to_list() for x in recs])*1)
    return {i: (len(val), sum(val), round(100*sum(val)/max(1,len(val)),2)) for i,val in tmp.items()}

In [19]:
results = {}
for K in [1, 5, 10]:
    results[K] = one_N(0,K)

100%|██████████| 831/831 [16:12<00:00,  1.17s/it]


 Considerando apenas os portfólios 2 e 3, numerados abaixo como 1 e 2 (resp.).

In [20]:
def show_results(results):
    return pd.DataFrame([[K]+flat([list(a) for a in r.values()]) for K,r in results.items()], columns=['K', 'Total_1', 'Acertos_1', 'Percentual_1 (%)', 'Total_2', 'Acertos_2', 'Percentual_2 (%)'])

In [21]:
show_results(results)

,K,Total_1,Acertos_1,Percentual_1 (%),Total_2,Acertos_2,Percentual_2 (%)
0,1,566,136,24.03,265,65,24.53
1,5,566,369,65.19,265,168,63.40
2,10,566,458,80.92,265,208,78.49


## Recomendações S-K
- Entra `S` IDs e saem `K` recomendações.

In [22]:
from tqdm import tqdm

def more_N(N,S,K):
    tmp = {1: [], 2: []}
    for n in tqdm(range(N)):
        for P in portfolios.P.unique():
            condP = portfolios.P == P
            df_tmp = portfolios.loc[condP].sample(n=S)
            recs = model.For(df_tmp.id.to_list(),k=K)
            tmp[P].append(any([x in portfolios.loc[condP].id.to_list() for x in recs])*1)
    return {i: (len(val), sum(val), round(100*sum(val)/max(1,len(val)),2)) for i,val in tmp.items()}

### 5-K

In [23]:
results2 = {}
for K in [1, 5, 10]:
    results2[K] = more_N(100,5,K)

100%|██████████| 100/100 [19:36<00:00, 11.76s/it]


In [24]:
show_results(results2)

,K,Total_1,Acertos_1,Percentual_1 (%),Total_2,Acertos_2,Percentual_2 (%)
0,1,100,23,23.0,100,23,23.0
1,5,100,70,70.0,100,70,70.0
2,10,100,91,91.0,100,86,86.0


## 10-K

In [25]:
results3 = {}
for K in [1, 5, 10]:
    results3[K] = more_N(100,10,K)

 97%|█████████▋| 97/100 [39:32<01:13, 24.46s/it]


KeyboardInterrupt: 

In [ ]:
show_results(results3)

In [27]:
model.model.M.shape

(462298, 62)